# 5. LangChain Expression Language（LCEL）徹底解説


In [11]:
import os
from dotenv import load_dotenv

current_working_directory = os.getcwd()
dotenv_path = os.path.join(current_working_directory, 'rag_ai_agent_book', '.env')

print(f"現在の作業ディレクトリ: {current_working_directory}")
print(f"参照する.envパス: {dotenv_path}")

load_dotenv(dotenv_path=dotenv_path)

# 環境変数がロードされたか確認し、必要であればos.environに設定
api_key = os.getenv("OPENAI_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")

if api_key is None:
    print("OPENAI_API_KEY 環境変数が設定されていません。")
    print("rag_ai_agent_book/ディレクトリに.envファイルを作成し、OpenAI APIキーを設定してください。")
    print("例: OPENAI_API_KEY=\"sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx\"")
else:
    os.environ["OPENAI_API_KEY"] = api_key
    print("OPENAI_API_KEY が正常に設定されました。")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key if langchain_api_key else ""
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

現在の作業ディレクトリ: /Users/kenichi/Projects
参照する.envパス: /Users/kenichi/Projects/rag_ai_agent_book/.env
OPENAI_API_KEY が正常に設定されました。


## 5.1. Runnable と RunnableSequence―LCEL の最も基本的な構成要素


In [12]:
# LangChainの出力パーサー（文字列用）をインポート
from langchain_core.output_parsers import StrOutputParser
# LangChainのチャットプロンプトテンプレートをインポート
from langchain_core.prompts import ChatPromptTemplate
# OpenAIのチャットモデル（LangChainラッパー）をインポート
from langchain_openai import ChatOpenAI

# システムメッセージとユーザーメッセージからプロンプトテンプレートを作成
prompt = ChatPromptTemplate.from_messages(
    [
        # システムメッセージ：ユーザーが入力した料理を京都風にアレンジしたレシピを考えるよう指示
        ("system", "ユーザーが入力した料理の京都風にアレンジしたレシピを考えてください。"),
        # ユーザーメッセージ：{dish}は後で入力される料理名に置き換わる
        ("human", "{dish}")
    ]
)

# OpenAIのgpt-4o-miniモデルを温度0で初期化（出力の多様性を抑える）
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# モデルの出力を文字列としてパースするためのパーサーを初期化
output_parser = StrOutputParser()

In [13]:
# プロンプトテンプレートに「フレンチトースト」を入力し、プロンプト値を生成
prompt_value = prompt.invoke({"dish": "フレンチトースト"})
# 生成したプロンプト値をOpenAIチャットモデルに渡してAIメッセージを取得
ai_message = model.invoke(prompt_value)
# AIメッセージを文字列としてパース（整形）する
output = output_parser.invoke(ai_message)
# 最終的な出力（京都風レシピ）を表示
print(output)

京都風フレンチトーストのレシピをご紹介します。抹茶や黒蜜、季節の果物を使って、和のテイストを取り入れたアレンジです。

### 材料（2人分）
- 食パン（厚切り） 2枚
- 卵 2個
- 牛乳 100ml
- 砂糖 大さじ1
- 抹茶パウダー 小さじ1（お好みで）
- バター 適量
- 黒蜜 適量
- 季節の果物（いちご、バナナ、柿など） 適量
- 粉砂糖（お好みで） 適量

### 作り方
1. **卵液を作る**: ボウルに卵を割り入れ、牛乳、砂糖、抹茶パウダーを加え、よく混ぜます。抹茶はお好みで調整してください。

2. **パンを浸す**: 食パンを卵液に浸し、両面がしっかりと液を吸収するようにします。約5分ほど置いておくと良いでしょう。

3. **焼く**: フライパンにバターを熱し、浸した食パンを中火で焼きます。両面がこんがりときつね色になるまで焼きます（約3〜4分ずつ）。

4. **盛り付け**: 焼き上がったフレンチトーストを皿に盛り、上に季節の果物をトッピングします。黒蜜をたっぷりかけ、お好みで粉砂糖を振りかけて完成です。

### 提案
- 抹茶の代わりに、きな粉を使っても美味しいです。
- フレンチトーストの上に、あんこやクリームチーズを添えると、さらに和の風味が楽しめます。

この京都風フレンチトーストは、和の素材を取り入れた新しい味わいを楽しむことができます。ぜひお試しください！


In [14]:
# プロンプト、モデル、出力パーサーをパイプ（|）で連結してチェーンを作成
chain = prompt | model | output_parser
# チェーンに「グラタン」という料理名を入力して京都風レシピを生成
output = chain.invoke({"dish": "グラタン"})
# 生成された京都風レシピを表示
print(output)

京都風グラタンのレシピをご紹介します。京都の食材や風味を取り入れた、和のテイストを感じるグラタンです。

### 材料（4人分）
- マカロニ：200g
- 鶏むね肉：200g（または鶏ささみ）
- ほうれん草：1束
- しいたけ：4〜5個
- 京揚げ（厚揚げ）：1枚
- 牛乳：400ml
- 生クリーム：100ml
- バター：30g
- 小麦粉：30g
- 塩：適量
- こしょう：適量
- みりん：大さじ1
- だしの素：小さじ1（または和風だし）
- とろけるチーズ：適量
- 青ねぎ（飾り用）：適量

### 作り方
1. **下準備**：
   - マカロニは塩を加えたお湯で茹で、表示時間より1分短く茹でて水を切ります。
   - 鶏肉は一口大に切り、塩、こしょう、みりんで下味をつけます。
   - ほうれん草はさっと茹でて水にさらし、絞って食べやすい大きさに切ります。
   - しいたけは薄切り、京揚げは食べやすい大きさに切ります。

2. **具材を炒める**：
   - フライパンにバターを溶かし、鶏肉を炒めます。鶏肉が白くなったら、しいたけと京揚げを加えてさらに炒めます。

3. **ホワイトソースを作る**：
   - 別の鍋にバターを溶かし、小麦粉を加えてよく混ぜ、少し色がつくまで炒めます。
   - 牛乳と生クリームを少しずつ加え、ダマにならないように混ぜながら煮ます。
   - だしの素を加え、塩、こしょうで味を調えます。

4. **グラタンを組み立てる**：
   - 炒めた具材とほうれん草をホワイトソースに加え、よく混ぜます。
   - 茹でたマカロニを加え、全体をよく混ぜ合わせます。

5. **焼く**：
   - グラタン皿に具材を入れ、とろけるチーズをたっぷりのせます。
   - 予熱したオーブンで180℃で約20〜25分、表面がこんがりと焼き色がつくまで焼きます。

6. **仕上げ**：
   - 焼き上がったら、青ねぎを散らして彩りを添えます。

### 提供
熱々のうちにお召し上がりください。京都の風味を感じる、和風グラタンの完成です！お好みで、七味唐辛子を振りかけても美味しいです。


### Runnable の実行方法―invoke・stream・batch


In [15]:
# チェーン（プロンプト、モデル、出力パーサー）をパイプで連結して作成
chain = prompt | model | output_parser
# 「ビザ」という料理名を入力し、京都風レシピをストリームで逐次生成
for chunk in chain.stream({"dish": "ビザ"}):
    # 生成されたテキストのチャンクをそのまま出力（改行なし、即時出力）
    print(chunk, end="", flush=True)

京都風のビザをアレンジしたレシピをご紹介します。伝統的なイタリアンピザに、京都の食材や風味を取り入れた一品です。

### 京都風ピザ「京野菜と抹茶のピザ」

#### 材料（2人分）
- ピザ生地（市販または手作り）: 1枚
- モッツァレラチーズ: 150g
- 京野菜（九条ネギ、賀茂ナス、壬生菜など）: 適量
- 抹茶パウダー: 小さじ1
- オリーブオイル: 大さじ1
- 塩: 少々
- 黒胡椒: 少々
- みりん: 小さじ1
- しょうゆ: 小さじ1（お好みで）

#### 作り方
1. **準備**: オーブンを220℃に予熱します。ピザ生地を薄く伸ばして、オーブンシートの上に置きます。

2. **野菜の下ごしらえ**: 京野菜を洗い、九条ネギは斜め切り、賀茂ナスは薄切り、壬生菜はざく切りにします。

3. **トッピングの準備**: フライパンにオリーブオイルを熱し、賀茂ナスを軽く炒め、塩と黒胡椒で味付けします。九条ネギと壬生菜も加え、さっと炒めます。

4. **ピザの組み立て**: ピザ生地の上にモッツァレラチーズを均等に散らし、その上に炒めた京野菜をのせます。

5. **抹茶の風味**: 抹茶パウダーを全体にふりかけ、さらにオリーブオイルを少し垂らします。

6. **焼く**: 予熱したオーブンで約10〜12分、チーズが溶けて生地がこんがりと焼き色がつくまで焼きます。

7. **仕上げ**: 焼き上がったら、お好みでみりんとしょうゆを軽くかけて、風味を引き立てます。

8. **サーブ**: 切り分けて、お皿に盛り付けて完成です。抹茶の香りと京野菜の旨味が楽しめる、京都風のピザをお楽しみください。

このレシピは、京都の食材を使い、抹茶の風味を加えることで、独特の味わいを楽しむことができます。ぜひお試しください！

In [17]:
chain=prompt|model|output_parser

outputs=chain.batch([{"dish":"ステーキ"},{"dish":"うなぎ"}])
print(outputs)
# for output in outputs:
# 	print(output)

['京都風のステーキアレンジレシピをご紹介します。京都の食材や風味を取り入れた、和のテイストを感じるステーキです。\n\n### 京都風ステーキのレシピ\n\n#### 材料（2人分）\n- 牛肉（サーロインまたはリブロース） 300g\n- みりん 2 tbsp\n- 醤油 2 tbsp\n- 砂糖 1 tsp\n- 生姜（すりおろし） 1 tsp\n- にんにく（すりおろし） 1 tsp\n- 青ねぎ（小口切り） 適量\n- ほうれん草（おひたし用） 1束\n- しいたけ（または舞茸） 2-3個\n- ごま油 1 tbsp\n- 七味唐辛子（お好みで） 適量\n\n#### 作り方\n1. **マリネ液の準備**: ボウルにみりん、醤油、砂糖、生姜、にんにくを混ぜ合わせ、マリネ液を作ります。\n\n2. **牛肉のマリネ**: 牛肉をマリネ液に漬け込み、30分から1時間ほど冷蔵庫で寝かせます。これにより、肉が柔らかくなり、風味が増します。\n\n3. **ほうれん草のおひたし**: ほうれん草をさっと茹でて冷水にさらし、水気を切ります。軽く絞ってから、醤油を少々かけておひたしにします。\n\n4. **しいたけのソテー**: フライパンにごま油を熱し、スライスしたしいたけを加えて炒めます。塩を少々振りかけ、香ばしくなるまで炒めます。\n\n5. **ステーキの焼き方**: マリネした牛肉をフライパンで中火で焼きます。お好みの焼き加減に仕上げたら、焼き上がった肉をアルミホイルで包み、数分休ませます。\n\n6. **盛り付け**: 休ませたステーキをスライスし、皿に盛ります。周りにほうれん草のおひたしとしいたけを添え、青ねぎを散らします。お好みで七味唐辛子を振りかけて完成です。\n\n### 提案\nこの京都風ステーキは、和のテイストを取り入れた一品で、特にご飯との相性が抜群です。お酒と一緒に楽しむのもおすすめです。京都の風情を感じながら、ぜひお楽しみください！', '京都風のうなぎ料理として「京風うなぎの蒲焼き」をご提案します。京都の伝統的な味付けや食材を取り入れたレシピです。\n\n### 京風うなぎの蒲焼き\n\n#### 材料（2人分）\n- うなぎの蒲焼き（冷凍または生） 2尾\n- みりん 50ml\n- しょうゆ 50ml\n- 砂糖 1 table

### LCEL の「|」で様々な Runnable を連鎖させる


In [ ]:
# StrOutputParserをインポート（モデル出力を文字列としてパースするためのクラス）
from langchain_core.output_parsers import StrOutputParser
# ChatPromptTemplateをインポート（チャット形式のプロンプトを作成するためのクラス）
from langchain_core.prompts import ChatPromptTemplate
# ChatOpenAIをインポート（OpenAIのチャットモデルを利用するためのクラス）
from langchain_openai import ChatOpenAI

# OpenAIのチャットモデル（gpt-4o-mini）を初期化（温度は0で決定的な出力に設定）
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# モデル出力を文字列としてパースするパーサーを初期化
output_parser = StrOutputParser()

# ステップバイステップで回答するプロンプトを作成
cot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーの質問にステップバイステップで回答してください"),
        ("human", "{question}")
    ]
)
# プロンプト、モデル、出力パーサーをパイプで連結しチェーンを作成
cot_chain = cot_prompt | model | output_parser

# ステップバイステップの回答から結論だけを抽出するプロンプトを作成
summarize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ステップバイステップで考えた回答から結論だけ抽出してください"),
        ("human", "{text}"),
    ]
)
# プロンプト、モデル、出力パーサーをパイプで連結しサマライズ用チェーンを作成
summarize_chain = summarize_prompt | model | output_parser

# ステップバイステップ回答チェーンとサマライズチェーンを連結し、複合チェーンを作成
cot_summarize_chain = cot_chain | summarize_chain
# 複合チェーンに「10+2*3」という質問を入力し、結論のみを抽出
output = cot_summarize_chain.invoke({"question": "10+2*3"})
# 抽出された結論を表示
print(output)

結論: \(10 + 2 * 3 = 16\) です。


In [ ]:
# 個別の結果を確認する
cot_output=cot_chain.invoke({"question":"10+2*3"})
print(cot_output)
print("-------------------------")
summarize_output=summarize_chain.invoke({"text":cot_output})
print(summarize_output)

この計算をステップバイステップで行います。

1. **計算の順序を確認**: 数学では、掛け算と割り算は足し算と引き算よりも優先されます。したがって、まず掛け算を計算します。

2. **掛け算を計算**: 
   - \(2 * 3 = 6\)

3. **足し算を計算**: 
   - \(10 + 6 = 16\)

したがって、\(10 + 2 * 3 = 16\) です。
-------------------------
結論: \(10 + 2 * 3 = 16\) です。


## 5.2. RunnableLambda―任意の関数を Runnable にする


In [ ]:
# StrOutputParser をインポート（モデル出力を文字列としてパースするためのクラス）
from langchain_core.output_parsers import StrOutputParser
# ChatPromptTemplate をインポート（チャット形式のプロンプトを作成するためのクラス）
from langchain_core.prompts import ChatPromptTemplate
# ChatOpenAI をインポート（OpenAI のチャットモデルを利用するためのクラス）
from langchain_openai import ChatOpenAI

# チャットプロンプトを作成（system メッセージと human メッセージを定義）
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You ar a helpful assistant."),  # システムメッセージ（アシスタントの役割を指定）
        ("human", "{input}"),  # ユーザーからの入力を受け取るプレースホルダー
    ]
)
# OpenAI のチャットモデルを初期化（gpt-4o-mini、温度0で決定的な出力）
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# モデル出力を文字列としてパースするパーサーを初期化
output_parser = StrOutputParser()

#-----------------------------------------------------------
# RunnableLambda をインポート（任意の関数を Runnable としてチェーンに組み込むためのクラス）
from langchain_core.runnables import RunnableLambda

# 文字列を大文字に変換する関数を定義
def upper(text: str) -> str:
    return text.upper()

# プロンプト、モデル、出力パーサー、upper 関数をパイプで連結しチェーンを作成
chain = prompt | model | output_parser | RunnableLambda(upper)

# チェーンに入力を与えて実行し、AI の応答を取得
ai_message = chain.invoke({"input": "Hello!"})
# AI の応答（大文字変換後）を表示
print(ai_message)

HELLO! HOW CAN I ASSIST YOU TODAY?


### chain デコレーターを使った RunnableLambda の実装


In [ ]:
# chain デコレーターをインポート
from langchain_core.runnables import chain

# upper 関数を chain デコレーターでラップし、Runnable として利用可能にする
@chain
def upper(text:str)->str:
	return text.upper()

# prompt, model, output_parser, upper をパイプで連結しチェーンを作成
chain=prompt|model|output_parser|upper
# チェーンに入力を与えて実行し、AI の応答を取得
ai_message=chain.invoke({"input":"Hello!"})
# AI の応答（大文字変換後）を表示
print(ai_message)

HELLO! HOW CAN I ASSIST YOU TODAY?


### RunnableLambda への自動変換


In [ ]:
# @chainを利用せずにも使える
def upper(text: str) -> str:
    return text.upper()

chain = prompt | model | output_parser | upper
ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

HELLO! HOW CAN I ASSIST YOU TODAY?


### （コラム）独自の関数を stream に対応させたい場合


In [ ]:
from typing import Iterator


# イテレータとして受け取った文字列を大文字に変換して返す関数を定義
def upper(input_stream: Iterator[str]) -> Iterator[str]:
    # 入力ストリームから1行ずつ取り出す
    for text in input_stream:
        # 取り出した文字列を大文字に変換してyieldで返す
        yield text.upper()

chain = prompt | model | StrOutputParser() | upper

# chain.stream() を使って、AIチェーンのストリーム出力を逐次取得する
for chunk in chain.stream({"input": "Hello! who are you? and where you are? , who is american president?"}):
    # 取得した各チャンク（部分的な応答）を改行なしで即時出力する
    print(chunk, end="", flush=True)

HELLO! I AM AN AI LANGUAGE MODEL CREATED TO ASSIST WITH INFORMATION AND ANSWER QUESTIONS. I EXIST IN THE DIGITAL REALM AND DON'T HAVE A PHYSICAL LOCATION. AS OF MY LAST UPDATE IN OCTOBER 2023, THE PRESIDENT OF THE UNITED STATES IS JOE BIDEN. HE TOOK OFFICE ON JANUARY 20, 2021. IF YOU HAVE ANY MORE QUESTIONS, FEEL FREE TO ASK!

## 5.3. RunnableParallel―複数の Runnable を並列で処理する


In [36]:
# StrOutputParserをインポート（AI出力を文字列としてパースするためのクラス）
from langchain_core.output_parsers import StrOutputParser
# ChatPromptTemplateをインポート（チャット形式のプロンプトを作成するためのクラス）
from langchain_core.prompts import ChatPromptTemplate
# OpenAIのチャットモデルをインポート
from langchain_openai import ChatOpenAI

# gpt-4o-miniモデルを温度0で初期化
model=ChatOpenAI(model="gpt-4o-mini",temperature=0)
# 出力パーサーを初期化
output_parser=StrOutputParser()

# 楽観主義者用のプロンプトを作成
optimistic_prompt=ChatPromptTemplate.from_messages(
	[
		# システムメッセージ：楽観主義者として振る舞うよう指示
		("system","あなたは楽観主義者です。ユーザーの入力に対して楽観的な意見をください。ファクト情報も提示せよ"),
		# ユーザーからの入力（トピック）を受け取る
		("human","{topic}")
	]
)
# 楽観主義者チェーンを作成（プロンプト→モデル→出力パーサー）
optinistic_chain=optimistic_prompt|model|output_parser

# 悲観主義者用のプロンプトを作成
pessimistic_prompt=ChatPromptTemplate.from_messages(
	[
		# システムメッセージ：悲観主義者として振る舞うよう指示
		("system","あなたは悲観主義者です。ユーザーの入力に対して悲観的な意見をください。ファクト情報も提示せよ"),
		# ユーザーからの入力（トピック）を受け取る
		("human","{topic}")
	]
)
# 悲観主義者チェーンを作成（プロンプト→モデル→出力パーサー）
pessimistic_chain=pessimistic_prompt|model|output_parser

# pprintをインポート（出力を見やすく整形するため）
import pprint
# RunnableParallelをインポート（複数のRunnableを並列実行するためのクラス）
from langchain_core.runnables import RunnableParallel

# 楽観主義者・悲観主義者チェーンを並列実行するチェーンを作成
parallel_chain=RunnableParallel(
	{
		"optimistic_opinion":optinistic_chain,  # 楽観主義者の意見
		"pessimistic_opinion":pessimistic_chain  # 悲観主義者の意見
	}
)
# 並列チェーンにトピックを渡して実行し、出力を取得
output=parallel_chain.invoke({"topic":"生成AIの進化と、これからのSES事業の変化と勝ち筋について"})
# 出力を整形して表示
pprint.pprint(output)


{'optimistic_opinion': '生成AIの進化は、SES（システムエンジニアリングサービス）事業にとって非常にポジティブな影響をもたらすと考えられます。まず、生成AIは業務の効率化を促進し、エンジニアがよりクリエイティブな作業に集中できる環境を提供します。例えば、コードの自動生成やデバッグ支援、ドキュメント作成の自動化などが進むことで、エンジニアの負担が軽減され、より高い付加価値を生み出すことが可能になります。\n'
                       '\n'
                       'さらに、生成AIは新しいビジネスモデルやサービスの創出を促進します。SES事業者は、AIを活用した新しいソリューションを提供することで、顧客のニーズに応えることができ、競争力を高めることができます。例えば、AIを活用したデータ分析や予測モデルの構築など、より高度なサービスを提供することで、顧客の信頼を得ることができるでしょう。\n'
                       '\n'
                       '勝ち筋としては、以下のポイントが挙げられます：\n'
                       '\n'
                       '1. **AIスキルの習得**: '
                       'エンジニアが生成AIを活用するスキルを身につけることで、競争力を高めることができます。これにより、より複雑なプロジェクトにも対応できるようになります。\n'
                       '\n'
                       '2. **顧客ニーズの理解**: '
                       'AIを活用したサービスを提供する際には、顧客のニーズを深く理解し、それに基づいたソリューションを提案することが重要です。\n'
                       '\n'
                       '3. **コラボレーションの強化**: '
                       'AIを活用することで、チーム内のコラボレーションが促進され、より効率的なプロジェクト運営が可能になりま

### RunnableParallel の出力を Runnable の入力に連結する


In [37]:
# 2つの意見を統合するためのプロンプトを作成
synthesize_prompt=ChatPromptTemplate.from_messages(
	[
		# システムメッセージ：聡明で客観的な現実主義者として振る舞うよう指示
		("system","あなたは聡明で客観的な現実主義者です。２つの意見を総合的にまとめて、より現実的な意見を詳細にください"),
		# ユーザーからの入力：楽観的・悲観的意見を渡す
		("human","楽観的意見:{optimistic_opinion}\n悲観定期意見:{pessimistic_opinion}")
	]
)
# 楽観主義者・悲観主義者チェーンを並列実行し、その出力をsynthesize_promptに渡すチェーンを作成
synthesize_chain=(
	RunnableParallel(
		{
			"optimistic_opinion":optinistic_chain,
			"pessimistic_opinion":pessimistic_chain
		}
	)
	|synthesize_prompt
	|model
	|output_parser
)
# チェーンにトピックを渡して実行し、統合意見を取得
output=synthesize_chain.invoke({"topic":"生成AIの進化と、これからのSES事業の変化と勝ち筋について"})
# 統合意見を出力
print(output)

生成AIの進化に関する楽観的意見と悲観的意見を総合的に考慮すると、SES（システムエンジニアリングサービス）事業における現実的な見解は以下のようになります。

### 現実的な意見

生成AIの進化はSES事業に対して両面の影響を及ぼすことが予想されます。ポジティブな側面としては、業務の効率化や新しいサービスの創出が挙げられますが、同時に従来のエンジニアの需要が減少するリスクも存在します。このため、SES事業は変革の時期にあり、企業は適応力を求められています。

#### 1. **業務効率化と人材の再教育**
生成AIは、コードの自動生成やデバッグ支援、ドキュメント作成の自動化を通じて、エンジニアの負担を軽減し、よりクリエイティブな作業に集中できる環境を提供します。しかし、これにより従来のプログラミング業務が減少する可能性もあるため、企業はエンジニアに対してAIを活用するスキルの習得を促進し、再教育を行う必要があります。これにより、エンジニアは新たな価値を提供できるようになります。

#### 2. **競争の激化と淘汰のリスク**
SES事業においては、AIを活用できる企業が競争優位に立つ一方で、技術に追いつけない企業は淘汰されるリスクがあります。したがって、企業は生成AIを取り入れたビジネスモデルの構築や、AIと人間の協働を前提とした新しいサービスの提供に注力する必要があります。これにより、競争の中で生き残るための戦略を確立することが求められます。

#### 3. **長期的なパートナーシップの構築**
生成AIを活用することで、クライアントのニーズに迅速かつ的確に応えることが可能になります。これにより、SES事業はクライアントとの信頼関係を深め、長期的なパートナーシップを築くチャンスが増えるでしょう。ただし、これを実現するためには、企業は常に技術の進化に目を光らせ、柔軟に対応する姿勢が必要です。

### 結論
生成AIの進化はSES事業にとって新たな可能性をもたらす一方で、従来のビジネスモデルや人材の在り方に変革を迫るものでもあります。企業は、効率化を追求しつつも、エンジニアのスキル向上や新しいサービスの開発に注力することで、競争の中での生き残りを図る必要があります。未来は明るい可能性を秘めていますが、それを実現するためには現実的な戦略と柔軟な対応が不可

In [ ]:
# チェインの動作確認。同じことをやっているはず
synthesize_chain2=parallel_chain|synthesize_prompt|model|output_parser
output2=synthesize_chain2.invoke({"topic":"生成AIの進化と、これからのSES事業の変化と勝ち筋について"})
print(output2)


生成AIの進化に関する楽観的意見と悲観的意見を総合的に考慮すると、SES（システムエンジニアリングサービス）事業における現実的な見解は以下のようになります。

### 現実的な意見

生成AIの進化はSES事業に多くの機会を提供する一方で、リスクも伴います。以下のポイントを考慮することで、企業はこの変化に適応し、競争力を維持するための戦略を構築することができます。

1. **業務効率化と人材の再配置**:
   生成AIは業務の効率化を促進し、エンジニアがよりクリエイティブな作業に集中できる環境を提供しますが、同時に従来の業務が自動化されることで、エンジニアの需要が減少する可能性もあります。企業は、AIを活用して業務を効率化する一方で、エンジニアのスキルを再評価し、より高度な業務や新しい役割に再配置する必要があります。

2. **新しいビジネスモデルの模索**:
   生成AIを活用した新しいサービスの提供は、SES事業者にとって重要な戦略です。しかし、競争が激化する中で、差別化が難しくなる可能性があります。企業は、顧客のニーズを深く理解し、独自の価値を提供するためのイノベーションを追求することが求められます。

3. **教育とスキルの向上**:
   エンジニアが生成AIを効果的に活用するための教育やトレーニングプログラムの充実が不可欠です。AI技術の進化に追いつくためには、継続的な学習とスキルの向上が必要です。これにより、エンジニアは新しい技術に適応し、競争力を維持することができます。

4. **協働とパートナーシップの強化**:
   他の企業やスタートアップとの連携を強化することで、相互に補完し合い、より大きな価値を提供できるようになります。特に、AI技術を持つ企業とのパートナーシップは、SES事業者にとって重要な戦略となるでしょう。

5. **リスク管理と適応力**:
   SES事業は厳しい競争にさらされるため、企業はリスクを管理し、変化に柔軟に対応する能力を高める必要があります。市場の動向を常に把握し、迅速に戦略を見直すことが成功の鍵となります。

### 結論
生成AIの進化はSES事業に新たなチャンスをもたらす一方で、競争の激化や人材の再配置といった課題も伴います。企業は、効率化と人材の再配置、新しいビジネスモデルの模索、教育とスキルの向

### RunnableLambda との組み合わせ―itemgetter を使う例


In [ ]:
# operatorモジュールからitemgetter関数をインポート
from operator import itemgetter

# 辞書型でAIに関する情報を定義（トピックとタイトル）
dec_about_ai={"topic":"生成AIの進化について","title":"生成AIの反乱"}
# "topic"キーの値を取得するためのitemgetterオブジェクトを作成
topic_getter=itemgetter("topic")
# itemgetterを使って辞書から"topic"の値を取得
topic=topic_getter(dec_about_ai)
# 取得したトピックを出力
print(topic)


生成AIの進化について


In [ ]:
# operatorモジュールからitemgetter関数をインポートする
from operator import itemgetter

# ChatPromptTemplateを使って、AIへのプロンプト（指示文）を定義する
synthesize_prompt=ChatPromptTemplate.from_messages(
	[
		# systemメッセージとしてAIに「{topic}について2つの意見をまとめて、より現実的で示唆に富んだ説得力のある意見にしてください」と指示している
		(
			"system",
            "あなたは客観的AIです。{topic}について2つの意見をまとめて、より現実的で示唆に富んだ説得力のある意見にしてください"
		),
		(
			"human",
			"楽観的意見:{optimistic_opinion}\n悲観的意見:{pessimistic_opinion}"
		)
	]
)
# ChatPromptTemplate.from_messagesで、systemメッセージとhumanメッセージをリストで指定している
# systemメッセージではAIに対して「{topic}について2つの意見をまとめて、より現実的で示唆に富んだ説得力のある意見にしてください」と指示している
# humanメッセージでは「楽観的意見:{optimistic_opinion}\n悲観的意見:{pessimistic_opinion}」という形式で2つの意見を渡している

# 各意見チェーンとトピック取得関数をまとめて、プロンプト・モデル・パーサーにパイプでつなぐチェーンを作成する
synthesize_chain=(
	{
		"optimistic_opinion":optinistic_chain,
		"pessimistic_opinion":pessimistic_chain,
		"topic":itemgetter("topic")
	}
	|synthesize_prompt
	|model
	|output_parser
)
# synthesize_chainは辞書で各意見チェーンとitemgetter("topic")をまとめている
# その後、synthesize_prompt（プロンプトテンプレート）、model（AIモデル）、output_parser（出力パーサー）をパイプでつないでいる

# チェーンに「日本の少子高齢化について」というトピックを渡して実行し、結果を取得する
output=synthesize_chain.invoke({"topic":"日本の少子高齢化について"})
# synthesize_chain.invokeで{"topic":"日本の少子高齢化について"}という入力を渡している
# その結果をoutput変数に格納している

# 結果を出力する
print(output)
# print関数でoutputの内容を表示している


日本の少子高齢化は、確かに深刻な課題でありながら、同時に新たな機会をもたらす可能性も秘めています。この現象は、労働力不足や社会保障制度の持続可能性に対する懸念を引き起こしていますが、適切な対策を講じることで、社会全体の活力を高めるチャンスにもなり得ます。

まず、少子高齢化によって高齢者の割合が増加する中で、彼らの知識や経験を活かすことが重要です。高齢者が労働市場に参加することで、労働力不足を補い、企業にとっても貴重な人材となります。これにより、経済の活性化が期待できるでしょう。また、高齢者向けの新しいビジネスモデルやサービスが生まれることで、経済成長の新たな原動力となる可能性もあります。

一方で、少子化に対する対策も急務です。政府は保育所の整備や育児休暇の充実を進めており、これにより若い世代が安心して子どもを持つ環境を整えています。しかし、これらの施策が実際に効果を上げるためには、さらなる取り組みが必要です。例えば、働き方改革を進めることで、仕事と家庭の両立を支援し、若者が子育てをしやすい社会を実現することが求められます。

また、テクノロジーの進化も重要な要素です。AIやロボット技術の発展により、高齢者の生活をサポートする新しいサービスが増えてきています。これにより、高齢者がより自立した生活を送ることができ、社会全体の活力が向上するでしょう。

結論として、日本の少子高齢化は挑戦であると同時に、社会の変革や新しいビジネスチャンスを生むきっかけともなり得ます。悲観的な見方だけでなく、ポジティブな視点を持って、未来に向けた具体的な対策を講じていくことが重要です。社会全体で協力し、持続可能な未来を築くための取り組みを進めていく必要があります。


## 5.4. RunnablePassthrough―入力をそのまま出力する


In [1]:
# import os
# from google.colab import userdata
# os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

# GoogleColabのコードをローカルで使えるように改造した

import os
from dotenv import load_dotenv

# 既に.envが読み込まれていない場合のみ読み込む
if "TAVILY_API_KEY" not in os.environ:
    current_working_directory = os.getcwd()
    dotenv_path = os.path.join(current_working_directory, 'rag_ai_agent_book', '.env')
    load_dotenv(dotenv_path=dotenv_path)

tavily_api_key = os.getenv("TAVILY_API_KEY")

if tavily_api_key is None:
    print("TAVILY_API_KEY 環境変数が設定されていません。")
    print("rag_ai_agent_book/ディレクトリの.envファイルにTavily APIキーを追加してください。")
    print("例: TAVILY_API_KEY=\"tvly-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx\"")
else:
    os.environ["TAVILY_API_KEY"] = tavily_api_key
    print("TAVILY_API_KEY が正常に設定されました。")

TAVILY_API_KEY が正常に設定されました。


In [ ]:
# 中身の確認の為のコード。.envを修正したらカーネルの再起動が必要だった
print(current_working_directory)
print(dotenv_path)
print(tavily_api_key)

/Users/kenichi/Projects
/Users/kenichi/Projects/rag_ai_agent_book/.env
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


In [5]:
# ChatPromptTemplateとChatOpenAIを使ってプロンプトとモデルを定義している
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。
文脈: """
{context}
"""
質問: {question}
''')
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [6]:
# Tavilyの検索APIリトリーバーをk=3で初期化
from langchain_community.retrievers import TavilySearchAPIRetriever
retriever = TavilySearchAPIRetriever(k=3)

In [19]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# RunnablePassthrough()は、入力値をそのまま次の処理に渡すためのランナブルです。
# ここでは、"question"キーに入力された値（ユーザーからの質問）をそのままチェーンに流しています。
chain=(
	{"context":retriever,"question":RunnablePassthrough()}
	|prompt
	|model
	|StrOutputParser()
)

output=chain.invoke("埼玉県戸田市の明日の天気と最高気温を教えて")
print(output)

埼玉県戸田市の明日、7月14日（火）の天気は「雨時々曇」で、最高気温は28℃です。


### assign―RunnableParallel に値を追加する


In [21]:
import pprint

# assign()は、既存のランナブル（ここでは辞書）に新たなキーと値（ここでは"answer"）を追加するメソッドです。
# "answer"の値には、prompt | model | StrOutputParser() というチェーンを指定しており、
# これはpromptでプロンプトを生成し、modelでAIモデルに投げ、StrOutputParser()で文字列として出力をパースします。
# assign()を使うことで、元の入力情報（question, context）に加えて、AIの回答（answer）も含めた複合的な出力を得ることができます。
chain={
	"question":RunnablePassthrough(),
	"context":retriever,
}|RunnablePassthrough.assign(answer=prompt|model|StrOutputParser())

output=chain.invoke("アシックスのGEL-NIMBUS27の魅力と他商品に比べて優れているところを教えて")
pprint.pprint(output)

{'answer': 'アシックスのGEL-NIMBUS '
           '27の魅力は、安定性とクッション性が両立している点です。重量感があるものの、抜群の安心感を提供し、中〜長距離ランナーにとって非常に適したシューズです。また、ジョグやリカバリーに最良とされており、足がぐらつきにくく、路面をしっかり捉えることで膝や足首への負担を軽減します。これにより、長距離を走った後の疲労感も少なく感じられるため、他のシューズと比べて優れたパフォーマンスを発揮します。',
 'context': [Document(metadata={'title': '【レビュー】ASICS GEL‑Nimbus 27｜雲の上の履き心地！超厚底で ...', 'source': 'https://note.com/joyful_worm2025/n/n50e0c9747b63', 'score': 0.7274535, 'images': []}, page_content='**「安定性とクッション性が両立している」**：重量感があるものの、抜群の安心感が魅力 。 **「ジョグとリカバリーに最良」**：中〜長距離ランナーからの'),
             Document(metadata={'title': 'GEL-NIMBUS 27｜走る楽しさに、ココロはずませよう。 - ASICS', 'source': 'https://www.asics.com/jp/ja-jp/mk/running/nimbus', 'score': 0.67243975, 'images': []}, page_content='Missing: 魅力 ところ 教え'),
             Document(metadata={'title': '【徹底比較】アシックス「GEL-KAYANO 32」VS「GEL-NIMBUS 27 ...', 'source': 'https://runstagramer.com/2025/06/09/asics_kayano_vs_nimbus/', 'score': 0.6219319, 'images': []}, page_content='足がぐらつきにくく、常に路面をしっかり捉えているような安心感があります。膝や足首への負担が軽減され、長

#### ＜補足：pick ＞


In [ ]:
chain = (
    RunnableParallel(
        {
            "question": RunnablePassthrough(),
            "context": retriever,
        }
    )
    .assign(answer=prompt | model | StrOutputParser())
    .pick(["context", "answer"])
)
output = chain.invoke("東京の今日の天気は？")
print(output)

### （コラム）astream_events


In [ ]:
# Google Colabでは次のコードの「async」の箇所に「Use of "async" not allowed outside of async function」と表示されますが、エラーなく実行できます

In [22]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

async for event in chain.astream_events("東京の今日の天気は？", version="v2"):
    print(event, flush=True)

{'event': 'on_chain_start', 'data': {'input': '東京の今日の天気は？'}, 'name': 'RunnableSequence', 'tags': [], 'run_id': '2d52b46f-7fc1-454e-8e68-fee50456929e', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {}, 'name': 'RunnableParallel<context,question>', 'tags': ['seq:step:1'], 'run_id': '9e963d1a-614c-4364-add1-811a67bbd08b', 'metadata': {}, 'parent_ids': ['2d52b46f-7fc1-454e-8e68-fee50456929e']}
{'event': 'on_retriever_start', 'data': {'input': {'query': '東京の今日の天気は？'}}, 'name': 'TavilySearchAPIRetriever', 'tags': ['map:key:context'], 'run_id': 'a6b3ff38-3f88-4912-970c-40ac9f8f5021', 'metadata': {'ls_retriever_name': 'tavilysearchapi'}, 'parent_ids': ['2d52b46f-7fc1-454e-8e68-fee50456929e', '9e963d1a-614c-4364-add1-811a67bbd08b']}
{'event': 'on_chain_start', 'data': {}, 'name': 'RunnablePassthrough', 'tags': ['map:key:question'], 'run_id': 'bcd749d9-aca1-404f-ba25-20416759273d', 'metadata': {}, 'parent_ids': ['2d52b46f-7fc1-454e-8e68-fee50456929e', '9e963d1a-614c-4364-

In [23]:
async for event in chain.astream_events("東京の今日の天気は？", version="v2"):
    event_kind = event["event"]

    if event_kind == "on_retriever_end":
        print("=== 検索結果 ===")
        documents = event["data"]["output"]
        for document in documents:
            print(document)

    elif event_kind == "on_parser_start":
        print("=== 最終出力 ===")

    elif event_kind == "on_parser_stream":
        chunk = event["data"]["chunk"]
        print(chunk, end="", flush=True)

=== 検索結果 ===
page_content='# tenki.jp 雨雲レーダー) 天気図 PM2.5分布予測 地震情報 日直予報士 熱中症情報 東京都の天気 ### 05月29日(木) 東京都の天気 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 晴時々曇 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 最新の天気履歴 渋谷区 曇のち雨 豊島区 曇のち雨 江東区 曇のち雨 港区 曇のち雨 大田区 曇のち雨 府中市 曇のち雨 奥多摩町 曇のち雨 関東・甲信 ### 気象予報士のポイント解説(日直予報士) newマーク 関東甲信　今日29日は天気下り坂　昼頃から所々で雨雲が湧く　夜は広く雨に 関東甲信　今日29日は天気下り坂　昼頃から所々で雨雲が湧く　夜は広く雨に newマーク 今日29日は気圧が低下　沖縄や九州から近畿は影響度「大」の所も　頭痛など注意 今日29日は気圧が低下　沖縄や九州から近畿は影響度「大」の所も　頭痛など注意 今日29日　九州から関東は次第に雨　東北と北海道は晴れて気温上昇　真夏日も 今日29日　九州から関東は次第に雨　東北と北海道は晴れて気温上昇　真夏日も ### 東京都各地の天気 #### 東京23区 #### 多摩 #### 伊豆諸島北部(大島) #### 伊豆諸島南部(八丈島) #### 小笠原諸島(父島) ### おすすめ記事 LINEの友達追加 ### 天気ガイド 雨雲 ### 注目の情報 アプリに便利なサブスクプラン開始 「tenki.jpライト」なら現在地の雨雲接近通知が受け取れる！ 新サービス「気圧予報」 気圧変化を確認して、頭痛やめまい、倦怠感といった症状に備えましょう。 X（旧Twitter） tenki.jpの公式X（旧Twitter） 最新の気象・防災情報や、生活に役立つ情報を毎日リアルタイムに配信中！ 天気予報 観測 防災情報 天気図 指数情報 レジャー天気 季節特集 天気ニュース X(旧：Twitter) Youtube Facebook Instagram LINEの友達追加 tenki.jp tenki.jp tenki.jp 登山天気 tenki.jp 登山天気 全国のコンテンツ tenki.jpトップ 天気予報 観測 防災情報 天気図' metadata={'titl

### （コラム）Chat history と Memory


In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

chain = prompt | model | StrOutputParser()

In [27]:
from langchain_community.chat_message_histories import SQLChatMessageHistory


def respond(session_id: str, human_message: str) -> str:
    chat_message_history = SQLChatMessageHistory(
        session_id=session_id, connection="sqlite:///sqlite.db"
    )

    ai_message = chain.invoke(
        {
            "chat_history": chat_message_history.get_messages(),
            "input": human_message,
        }
    )

    chat_message_history.add_user_message(human_message)
    chat_message_history.add_ai_message(ai_message)

    return ai_message

In [28]:
from uuid import uuid4

session_id = uuid4().hex

output1 = respond(
    session_id=session_id,
    human_message="こんにちは！私はジョンと言います！",
)
print(output1)

output2 = respond(
    session_id=session_id,
    human_message="私の名前が分かりますか？",
)
print(output2)

こんにちは、ジョンさん！お会いできて嬉しいです。今日はどんなことをお話ししましょうか？
はい、あなたの名前はジョンさんです。何か特別なことについてお話ししたいことがありますか？
